In [215]:
### Data matrices

# i -> index of products (p1,p2,p3)
# j -> index of resources (r1, r2, r3)
# k -> index of market ( lo2, med , high)

Demand_states = ["l", "m", "h"]

# selling price with index i for each product
Price = [60 ,40 ,10]  

# resource recipe with index i,j where i stands for product and j for resource
r = [[8, 6, 1], 
     [2, 1.5, 0.5], 
     [4 ,2, 1.5]] #ise index i for prod, j for resource [j,i]
c = [2, 5.2, 4]

# demand for each product at different market conditions, each column is one product, each row is one market condition
Demand = [[50, 150, 250],
        [20, 110, 250],
        [200, 225, 500]]  #k,i

# probabbility of each market condition, use index k 
Prob = [0.3, 0.4, 0.3]

In [216]:
import gurobipy as gb
from gurobipy import GRB

In [217]:
PPP = gb.Model("Postponement Problem_LP & Uncertainty")

In [218]:
# 
I = len(c)
K = len(Price)
J = len(Prob)


# decision variables 

R = PPP.addVars(I, lb = 0, vtype = GRB.CONTINUOUS, name = ["R"+str(i) for i in range(J)])
P = PPP.addVars(K, lb = 0, vtype = GRB.CONTINUOUS, name = ["P"+str(k) for k in range(I)])
S = PPP.addVars(J, K, lb = 0, vtype = GRB.CONTINUOUS, name = ["Sales of Product  "+str(k)+" in demand state "+str(j) for k in ['l','m','h'] for j in range(J)])

In [219]:
# Objective

sales = sum(pr[j]*S[k,j]*sp[k] for k in range(K) for j in range(J))

cost = sum(cr[i]*R[i] for i in range(I))
exp = sales - cost

PPP.setObjective(exp, GRB.MAXIMIZE)

In [220]:
# Constraints

# Non negativity is defined in definition

# resource constraint
for j in range(J):
    PPP.addConstr(sum(P[i]*r[j][i] for i in range(I)) <= R[j])

# Sales cannot be more than demand
for i in range(I):
    for k in range(K):
        PPP.addConstr(S[k,i] <= d[k][i]) 


# sales cannot be more than production
for k in range(K):
    for j in range(J):
        PPP.addConstr(S[k,j] <= P[k] )

In [221]:
PPP.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 15 columns and 39 nonzeros
Model fingerprint: 0xd5963a1d
Coefficient statistics:
  Matrix range     [5e-01, 8e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 5e+02]
Presolve removed 12 rows and 3 columns
Presolve time: 0.01s
Presolved: 9 rows, 12 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.4220000e+03   6.625000e+01   0.000000e+00      0s
       3    1.1420000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.142000000e+03


In [222]:
PPP.objVal

1142.0

In [223]:
for v in PPP.getVars():
    print(v.varname, "=", round(v.x))

R0 = 1060
R1 = 265
R2 = 420
P0 = 50
P1 = 110
P2 = 0
Sales of Product  l in demand state 0 = 50
Sales of Product  l in demand state 1 = 50
Sales of Product  l in demand state 2 = 50
Sales of Product  m in demand state 0 = 20
Sales of Product  m in demand state 1 = 110
Sales of Product  m in demand state 2 = 110
Sales of Product  h in demand state 0 = 0
Sales of Product  h in demand state 1 = 0
Sales of Product  h in demand state 2 = 0


In [214]:
S

{(0, 0): <gurobi.Var Sales of Product  0 in demand state l (value 20.0)>,
 (0, 1): <gurobi.Var Sales of Product  1 in demand state l (value 110.0)>,
 (0, 2): <gurobi.Var Sales of Product  2 in demand state l (value 0.0)>,
 (1, 0): <gurobi.Var Sales of Product  0 in demand state m (value 20.0)>,
 (1, 1): <gurobi.Var Sales of Product  1 in demand state m (value 110.0)>,
 (1, 2): <gurobi.Var Sales of Product  2 in demand state m (value 0.0)>,
 (2, 0): <gurobi.Var Sales of Product  0 in demand state h (value 20.0)>,
 (2, 1): <gurobi.Var Sales of Product  1 in demand state h (value 110.0)>,
 (2, 2): <gurobi.Var Sales of Product  2 in demand state h (value 0.0)>}